In [1]:
import re
import time
import pandas as pd
import xarray as xr
from numpy import ones
from numpy.linalg import cholesky
from pandas_plink import read_plink1_bin
from limix.qc import quantile_gaussianize

In [2]:
import cellregmap 
cellregmap 

<module 'cellregmap' from '/hps/nobackup/stegle/users/acuomo/git_repos/CellRegMap/cellregmap/__init__.py'>

In [3]:
from cellregmap import estimate_betas

In [4]:
input_files_dir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/input_files/"

In [5]:
## sample mapping file
## this file will map cells to donors 
## it will also only include donors we have single-cell data for (a subset of all of HipSci donors)
sample_mapping_file = input_files_dir+"sample_mapping_file.csv"
sample_mapping = pd.read_csv(sample_mapping_file, dtype={"genotype_individual_id": str, "phenotype_sample_id": str})

In [6]:
## genotype_individual_id are donor IDs, as found in the genotype matrix (G) and GRM covariance (K)
## phenotype_sample_id are cell IDs, as found in the scRNA-seq phenotype vector (y) and cell context covariance (C)
sample_mapping.head()

,genotype_individual_id,phenotype_sample_id
0,HPSI0114i-joxm_1,21843_1#10
1,HPSI0314i-fafq_1,21843_1#100
2,HPSI0314i-fafq_1,21843_1#101
3,HPSI1013i-wuye_2,21843_1#102
4,HPSI0114i-joxm_1,21843_1#103


In [7]:
## extract unique individuals
donors = sample_mapping["genotype_individual_id"].unique()
donors.sort()
print("Number of unique donors: {}".format(len(donors)))

Number of unique donors: 126


In [8]:
############################################
################ Kinship matrix ############
############################################

In [9]:
## read in GRM (genotype relationship matrix; kinship matrix)
kinship_folder="/hps/nobackup/hipsci/scratch/genotypes/imputed/2017-03-27/Full_Filtered_SNPs_Plink-F/"
kinship_file=kinship_folder+"hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.kinship"
K = pd.read_csv(kinship_file, sep="\t", index_col=0)
assert all(K.columns == K.index) #symmetric matrix, donors x donors

In [10]:
K = xr.DataArray(K.values, dims=["sample_0", "sample_1"], coords={"sample_0": K.columns, "sample_1": K.index})
K = K.sortby("sample_0").sortby("sample_1")
donors = sorted(set(list(K.sample_0.values)).intersection(donors))
print("Number of donors after kinship intersection: {}".format(len(donors)))

Number of donors after kinship intersection: 125


In [11]:
## subset to relevant donors
K = K.sel(sample_0=donors, sample_1=donors)
assert all(K.sample_0 == donors)
assert all(K.sample_1 == donors)

In [12]:
## and decompose such as K = hK @ hK.T (using Cholesky decomposition)
hK = cholesky(K.values)
hK = xr.DataArray(hK, dims=["sample", "col"], coords={"sample": K.sample_0.values})
assert all(hK.sample.values == K.sample_0.values)

In [13]:
del K
print("Sample mapping number of rows BEFORE intersection: {}".format(sample_mapping.shape[0]))
## subsample sample mapping file to donors in the kinship matrix
sample_mapping = sample_mapping[sample_mapping["genotype_individual_id"].isin(donors)]
print("Sample mapping number of rows AFTER intersection: {}".format(sample_mapping.shape[0]))

Sample mapping number of rows BEFORE intersection: 34256
Sample mapping number of rows AFTER intersection: 33964


In [14]:
############################################
##### expand from donors to cells ##########

In [15]:
## use sel from xarray to expand hK (using the sample mapping file)
hK_expanded = hK.sel(sample=sample_mapping["genotype_individual_id"].values)
assert all(hK_expanded.sample.values == sample_mapping["genotype_individual_id"].values)

In [16]:
######################################
############### Genotypes ############
######################################

In [17]:
## read in genotype file (plink format)
plink_folder = "/hps/nobackup/hipsci/scratch/genotypes/imputed/2017-03-27/Full_Filtered_SNPs_Plink/"
plink_file = plink_folder+"hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.bed"
G = read_plink1_bin(plink_file)

Mapping files: 100%|██████████| 3/3 [06:11<00:00, 123.82s/it]


In [18]:
#############################
###### SNP selection

In [19]:
# # filter file (columns: snp_id, gene)
# revision_folder = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/"
# fvf_filename = revision_folder+"fvf_new_outliers.csv"
# fvf = pd.read_csv(fvf_filename, index_col = False)
# fvf.head(2)

,Unnamed: 0,feature,snp_id
0,397102,ENSG00000105426_PTPRS,19_5181195_C_T
1,397110,ENSG00000105426_PTPRS,19_5184684_C_T


In [26]:
# filter file (columns: snp_id, gene)
endo_eqtl_file = input_files_dir+"endodiff_eqtl_allconditions_FDR10pct.csv"
endo_eqtl = pd.read_csv(endo_eqtl_file, index_col = False)
endo_eqtl["chrom"] = [int(i[:i.find("_")]) for i in endo_eqtl["snp_id"]]
endo_eqtl.head(2)

,snp_id,feature,stage,chrom
0,5_149826526_C_T,ENSG00000164587_RPS14,ips,5
1,11_57283988_C_T,ENSG00000134809_TIMM10,ips,11


In [27]:
fvf = endo_eqtl

In [28]:
genes = fvf['feature'].unique()
genes

array(['ENSG00000164587_RPS14', 'ENSG00000134809_TIMM10',
       'ENSG00000197728_RPS26', ..., 'ENSG00000132130_LHX1',
       'ENSG00000112559_MDFI', 'ENSG00000141012_GALNS'], dtype=object)

In [29]:
gene_name = "ENSG00000109625_CPZ"

In [30]:
# (1) gene name (feature_id)
# gene_name = genes[1]
trait_name = re.sub("_.*","",gene_name)
trait_name

'ENSG00000109625'

In [31]:
leads = fvf[fvf['feature']==gene_name]['snp_id'].unique()

In [32]:
G_sel = G[:,G['snp'].isin(leads)]

In [33]:
G_sel.shape

(1610, 2)

In [34]:
############################################
##### expand from donors to cells ##########

In [35]:
# expand out genotypes from cells to donors (and select relevant donors in the same step)
G_expanded = G_sel.sel(sample=sample_mapping["genotype_individual_id"].values)
assert all(hK_expanded.sample.values == G_expanded.sample.values)

/nfs/software/stegle/users/acuomo/conda-envs/struct-lmm2/lib/python3.8/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing with an out-of-order index is generating 2383 times more chunks
  return self.array[key]


In [36]:
G_expanded.shape

(33964, 2)

In [37]:
######################################
############## Phenotypes ############
######################################

In [38]:
# Phenotype (single-cell expression)
phenotype_file = input_files_dir+"phenotype.csv.pkl"
phenotype = pd.read_pickle(phenotype_file)
print("Phenotype shape BEFORE selection: {}".format(phenotype.shape))
phenotype = xr.DataArray(phenotype.values, dims=["trait", "cell"], coords={"trait": phenotype.index.values, "cell": phenotype.columns.values})
phenotype = phenotype.sel(cell=sample_mapping["phenotype_sample_id"].values)
print("Phenotype shape AFTER selection: {}".format(phenotype.shape))
assert all(phenotype.cell.values == sample_mapping["phenotype_sample_id"].values)

Phenotype shape BEFORE selection: (11231, 34256)
Phenotype shape AFTER selection: (11231, 33964)


In [39]:
# select gene
y = phenotype.sel(trait=gene_name)
# quantile normalise
y = quantile_gaussianize(y)

In [40]:
######################################
########## Cell contexts #############
######################################

In [41]:
# cells by MOFA factors (20)
C_file = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/mofa_logcounts_model_factors.csv"
C = pd.read_csv(C_file, index_col = 0)
C = xr.DataArray(C.values, dims=["cell", "pc"], coords={"cell": C.index.values, "pc": C.columns.values})
C = C.sel(cell=sample_mapping["phenotype_sample_id"].values)
assert all(C.cell.values == sample_mapping["phenotype_sample_id"].values)

In [42]:
# quantile normalise cell contexts
C = quantile_gaussianize(C)

In [34]:
######################################
############ Covariates ##############
######################################

In [43]:
n_cells = phenotype.shape[1]
W = ones((n_cells, 1))

In [44]:
y = y.values.reshape(y.shape[0],1)
y.shape

(33964, 1)

In [45]:
W

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [46]:
C.values[:,0:10].shape
C.values[:,0:10]

array([[-0.22698835,  0.82917178,  0.23411244, ...,  0.07390445,
         0.30870685, -0.66988754],
       [ 0.11928532,  2.94980516,  0.8196344 , ..., -0.3224365 ,
         1.85295159, -0.70238707],
       [ 0.05970302,  1.86963924,  0.02166209, ...,  0.1331965 ,
         1.00717123,  0.75346536],
       ...,
       [-0.39533548,  0.72850657, -1.20369879, ..., -0.20673893,
         0.31816383, -0.3551869 ],
       [-0.26984302,  1.04975257, -0.57387367, ...,  0.13543057,
         0.58636113, -0.12188718],
       [-0.38132928,  1.2401305 , -0.68501989, ...,  0.33271513,
         0.47799852,  0.28748933]])

In [47]:
start_time = time.time()
GG = G_expanded.values
print("--- %s seconds ---" % (time.time() - start_time))

--- 229.2718472480774 seconds ---


In [48]:
GG.shape

(33964, 2)

In [49]:
GG

array([[1., 2.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 2.],
       [2., 1.],
       [2., 1.]], dtype=float32)

In [50]:
start_time = time.time()
b = estimate_betas(y, W, C.values[:,0:10], G=GG, hK=hK_expanded)
print("--- %s seconds ---" % (time.time() - start_time))

--- 386.4946880340576 seconds ---


In [51]:
beta_G = b[0]                         # persistent effect (scalar)
beta_GxC = b[1][0]                    # GxC effects (vector)

In [52]:
beta_G

array([ 0.06425983, -0.0771264 ])

In [53]:
beta_G_df = pd.DataFrame({"chrom":G_expanded.chrom.values,
               "pv":beta_G,
               "variant":G_expanded.snp.values})
beta_G_df.head()

,chrom,pv,variant
0,4,0.064260,4_8567600_T_A
1,4,-0.077126,4_8599467_G_C


In [54]:
beta_GxC

array([[-0.03349241,  0.        ],
       [-0.22280189,  0.        ],
       [-0.16440142,  0.        ],
       ...,
       [ 0.03337502,  0.        ],
       [-0.06575692,  0.        ],
       [-0.086561  ,  0.        ]])

In [55]:
beta_GxC.shape

(33964, 2)

In [56]:
cells = phenotype["cell"].values
snps = G_expanded["variant"].values

beta_GxC_df = pd.DataFrame(data = beta_GxC, columns = snps, index = cells)
beta_GxC_df.head()

,4_4_8567600_T_A,4_4_8599467_G_C
21843_1#10,-0.033492,0.0
21843_1#100,-0.222802,0.0
21843_1#101,-0.164401,0.0
21843_1#102,0.015583,0.0
21843_1#103,-0.031477,0.0


In [ ]:
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/"
# outfile = outdir+"beta_GxC/"+str(gene_name)+".csv"

In [ ]:
# beta_GxC_df.to_csv(outfile)